In [2]:
import onnx
from onnx.tools import update_model_dims
import numpy as np
import onnx.helper as helper
from onnx import shape_inference, TensorProto
import sys

In [3]:
import torch
import torchvision
from transformers import AutoTokenizer, AutoModel

from pathlib import Path
from transformers.convert_graph_to_onnx import convert

In [1]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

inputs = tokenizer("Hello world!", return_tensors="pt")
outputs = model(**inputs)
print(outputs)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.1424,  0.1335, -0.1291,  ..., -0.3597, -0.0562,  0.3605],
         [-0.3506,  0.1042,  0.6244,  ..., -0.1761,  0.4834,  0.0644],
         [-0.2451, -0.1573,  0.6945,  ..., -0.5654, -0.0894, -0.1856],
         [-0.8248, -0.9119, -0.6561,  ...,  0.5074, -0.1939, -0.1659],
         [ 0.8767,  0.0352, -0.1233,  ...,  0.2720, -0.6369, -0.1585]]],
       grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[-8.9756e-01, -3.3040e-01, -7.6942e-01,  7.5799e-01,  4.6678e-01,
         -1.2035e-01,  9.1835e-01,  1.8087e-01, -7.2716e-01, -9.9991e-01,
         -4.4723e-01,  8.9104e-01,  9.6621e-01,  5.4915e-01,  9.4344e-01,
         -7.6605e-01, -6.0469e-01, -6.1654e-01,  4.0572e-01, -7.4644e-01,
          6.1739e-01,  9.9974e-01,  3.2990e-02,  2.5414e-01,  4.3106e-01,
          9.7732e-01, -8.4328e-01,  9.2297e-01,  9.4871e-01,  6.3994e-01,
         -7.3620e-01,  9.0957e-02, -9.7607e-01, -1.9115e-01, -8.1717e-01,
     

In [2]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [14]:
model = onnx.load("./models/bert-base/bert-base-cased.onnx")
oldnodes = [n for n in model.graph.node]
newnodes = oldnodes[0:125]
for n in model.graph.node:
    model.graph.node.remove(n)
model.graph.node.extend(newnodes)
onnx.save(model, "./models_out/bert-1-v2.onnx")

In [18]:
graph = helper.GraphProto()
graph.node.extend(newnodes)
model_def = helper.make_model(graph, producer_name='onnx-example')
onnx.save(model_def, "./models_out/bert-1-v3.onnx")

In [19]:
graph_def = helper.make_graph(
            ['Mul_123'],
            'Mul_123',
            inputs=['A','B'],  # 输入
            outputs=['C'],  # 输出
            )

TypeError: Not a cmessage

In [12]:
model = onnx.load("./models/vgg16.onnx")
for i in range(20):
    print(model.graph.initializer[i].dims)

[512, 512]
[512]
[512, 512]
[512]
[10, 512]
[10]
[64, 3, 3, 3]
[64]
[64, 64, 3, 3]
[64]
[128, 64, 3, 3]
[128]
[128, 128, 3, 3]
[128]
[256, 128, 3, 3]
[256]
[256, 256, 3, 3]
[256]
[256, 256, 3, 3]
[256]


In [4]:
input_path = './models/vgg16.onnx'
output_path = './models_out/vgg16_out.onnx'
input_names = ['100']
output_names = ['121']
onnx.utils.extract_model(input_path, output_path, input_names, output_names)

In [7]:
input_path = './models/bert-base/bert-base-cased.onnx'
output_path = './models_out/bert_out.onnx'
# input_names = ['input_ids', 'attention_mask', 'token_type_ids']
input_names = ['235']
output_names = ['351']
onnx.utils.extract_model(input_path, output_path, input_names, output_names)

ValidationError: Field 'shape' of type is required but missing.

In [6]:
model = onnx.load("./models/bert-base/bert-base-cased.onnx")
onnx.checker.check_model(model)

In [8]:
class Extractor:
    def __init__(self, model):  # type: (ModelProto) -> None
        self.model = onnx.shape_inference.infer_shapes(model)
        self.graph = self.model.graph
        self.wmap = self._build_name2obj_dict(self.graph.initializer)
        self.vimap = self._build_name2obj_dict(self.graph.value_info)

    @staticmethod
    def _build_name2obj_dict(objs):  # type: ignore
        return {obj.name: obj for obj in objs}

    def _collect_new_io_core(self, original_io, io_names_to_extract):  # type: ignore
        original_io_map = self._build_name2obj_dict(original_io)
        original_io_names = set(original_io_map.keys())
        s_io_names_to_extract = set(io_names_to_extract)
        io_names_to_keep = s_io_names_to_extract & original_io_names
        new_io_names_to_add = s_io_names_to_extract - original_io_names

        new_io_tensors = []
        for name in io_names_to_keep:
            new_io_tensors.append(original_io_map[name])
        for name in new_io_names_to_add:
            # activation become input or output
            new_io_tensors.append(self.vimap[name])

        # adjust sequence
        new_io_tensors_map = self._build_name2obj_dict(new_io_tensors)
        return [new_io_tensors_map[name] for name in io_names_to_extract]

    def _collect_new_inputs(self, names):  # type: (List[Text]) -> List[ValueInfoProto]
        return self._collect_new_io_core(self.graph.input, names)  # type: ignore

    def _collect_new_outputs(self, names):  # type: (List[Text]) -> List[ValueInfoProto]
        return self._collect_new_io_core(self.graph.output, names)  # type: ignore

    def _dfs_search_reachable_nodes(
            self,
            node_output_name,  # type: Text
            graph_input_names,  # type: List[Text]
            reachable_nodes,  # type: List[NodeProto]
    ):  # type: (...) -> None
        if node_output_name in graph_input_names:
            return
        for node in self.graph.node:
            if node in reachable_nodes:
                continue
            if node_output_name not in node.output:
                continue
            reachable_nodes.append(node)
            for name in node.input:
                self._dfs_search_reachable_nodes(name, graph_input_names, reachable_nodes)

    def _collect_reachable_nodes(
            self,
            input_names,  # type: List[Text]
            output_names,  # type: List[Text]
    ):  # type: (...) -> List[NodeProto]
        reachable_nodes = list()  # type: ignore
        for name in output_names:
            self._dfs_search_reachable_nodes(name, input_names, reachable_nodes)
        # needs to be topology sorted.
        nodes = [n for n in self.graph.node if n in reachable_nodes]
        return nodes

    def _collect_reachable_tensors(
            self,
            nodes,  # type: List[NodeProto]
    ):  # type: (...) -> Tuple[List[TensorProto], List[ValueInfoProto]]
        all_tensors_name = set()
        for node in nodes:
            for name in node.input:
                all_tensors_name.add(name)
            for name in node.output:
                all_tensors_name.add(name)

        initializer = [self.wmap[t] for t in self.wmap.keys() if t in all_tensors_name]
        value_info = [self.vimap[t] for t in self.vimap.keys() if t in all_tensors_name]
        assert(len(self.graph.sparse_initializer) == 0)
        assert(len(self.graph.quantization_annotation) == 0)
        return (initializer, value_info)

    def _make_model(
            self,
            nodes,  # type: List[NodeProto]
            inputs,  # type: List[ValueInfoProto]
            outputs,  # type: List[ValueInfoProto]
            initializer,  # type: List[TensorProto]
            value_info  # type: List[ValueInfoProto]
    ):  # type: (...) -> ModelProto
        name = 'Extracted from {' + self.graph.name + '}'
        graph = onnx.helper.make_graph(nodes, name, inputs, outputs, initializer=initializer,
                                      value_info=value_info)

        meta = {
            'ir_version': self.model.ir_version,
            'opset_imports': self.model.opset_import,
            'producer_name': 'onnx.utils.extract_model',
        }
        return onnx.helper.make_model(graph, **meta)

    def extract_model(
            self,
            input_names,  # type: List[Text]
            output_names,  # type: List[Text]
    ):  # type: (...) -> ModelProto
        inputs = self._collect_new_inputs(input_names)
        outputs = self._collect_new_outputs(output_names)
        nodes = self._collect_reachable_nodes(input_names, output_names)
        initializer, value_info = self._collect_reachable_tensors(nodes)
        model = self._make_model(nodes, inputs, outputs, initializer, value_info)

        return model

In [9]:
input_path = './models/bert-base/bert-base-cased.onnx'
output_path = './models_out/bert_out.onnx'
input_names = ['235']
output_names = ['351']
onnx.checker.check_model(input_path)
model = onnx.load(input_path)

e = Extractor(model)
extracted = e.extract_model(input_names, output_names)

In [10]:
onnx.save(extracted, output_path)

In [11]:
extracted = onnx.shape_inference.infer_shapes(extracted)
onnx.save(extracted, './models_out/bert_out_2.onnx')

In [3]:
! mkdir ./models/bert-text-classification

In [6]:
from pathlib import Path
from transformers.convert_graph_to_onnx import convert

# Handles all the above steps for you
p = {"num_labels": 2}
convert(framework="pt", model="bert-base-cased", output=Path("./models/bert-text-classification/bert-cls.onnx"), opset=11, pipeline_name="text-classification", **p)

ONNX opset version set to: 11
Loading pipeline (model: bert-base-cased, tokenizer: bert-base-cased)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Using framework PyTorch: 1.8.0
Found input input_ids with shape: {0: 'batch', 1: 'sequence'}
Found input token_type_ids with shape: {0: 'batch', 1: 'sequence'}
Found input attention_mask with shape: {0: 'batch', 1: 'sequence'}
Found output output_0 with shape: {0: 'batch'}
Ensuring inputs are in correct order
position_ids is not present in the generated input list.
Generated inputs order: ['input_ids', 'attention_mask', 'token_type_ids']


/home/slzhang/miniconda/envs/fedml/lib/python3.7/site-packages/transformers-4.8.1-py3.8.egg/transformers/modeling_utils.py:1974: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!


In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [8]:

from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

In [13]:
from datasets import load_dataset

In [1]:
from datasets import load_dataset
from datasets import list_datasets
datasets_list = list_datasets()
len(datasets_list)

1132

In [3]:
raw_datasets = load_dataset("imdb")

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to /home/slzhang/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a. Subsequent calls will reuse this data.


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [5]:

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [6]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000)) 
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000)) 
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [8]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [9]:

from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

In [10]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
)

/home/slzhang/miniconda/envs/fedml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/home/slzhang/miniconda/envs/fedml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/home/slzhang/miniconda/envs/fedml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/home/slzhang/miniconda/envs/fedml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)


In [11]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: zsl (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/home/slzhang/miniconda/envs/fedml/lib/python3.7/site-packages/torch/nn/modules/module.py:760: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/home/slzhang/miniconda/envs/fedml/lib/python3.7/site-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Step,Training Loss


KeyboardInterrupt: 